In [1]:
%reset -f

In [26]:
from pathlib import Path
from sssom.parsers import parse_sssom_json, parse_sssom_table
from sssom.writers import write_table, write_json

In [41]:
input_sssom = Path("__file__").resolve().parents[1] /"omim-ordo-do" / "boomer_input" / "mondo_exact" / "combined.sssom.tsv"
ofn_file = Path("__file__").resolve().parents[1] /"omim-ordo-do" / "boomer_output" / "mondo_exact.ofn"
ofn_json = Path("__file__").resolve().parents[1] /"omim-ordo-do" / "boomer_output" / "mondo_exact.json"
prefix_map = Path("__file__").resolve().parents[1] / "boomer_input"/ "prefix.yaml"
! robot convert -i {ofn_file} --format json -o {ofn_json}

In [33]:
input_msdf = parse_sssom_table(input_sssom)
input_msdf.df.head()


,subject_id,subject_label,predicate_id,object_id,mapping_justification,confidence
0,MONDO:0000001,disease or disorder,skos:exactMatch,SCTID:64572001,semapv:UnspecifiedMatching,0.99
1,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:377788,semapv:UnspecifiedMatching,0.99
2,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C2991,semapv:UnspecifiedMatching,0.99
3,MONDO:0000001,disease or disorder,skos:exactMatch,DOID:4,semapv:UnspecifiedMatching,0.99
4,MONDO:0000001,disease or disorder,skos:exactMatch,UMLS:C0012634,semapv:UnspecifiedMatching,0.99


In [35]:
meta= input_msdf.metadata

meta["curie_map"] = input_msdf.prefix_map

meta

{'license': 'https://w3id.org/sssom/license/unspecified',
 'mapping_set_id': 'https://w3id.org/sssom/mappings/74a8f3da-09b5-47ea-bedb-239ebe9d56d8',
 'curie_map': {'DOID': 'http://purl.obolibrary.org/obo/DOID_',
  'EFO': 'http://www.ebi.ac.uk/efo/EFO_',
  'HGNC': 'http://identifiers.org/hgnc/',
  'HP': 'http://purl.obolibrary.org/obo/HP_',
  'ICD10CM': 'http://purl.bioontology.org/ontology/ICD10CM/',
  'ICD10WHO': 'https://icd.who.int/browse10/2019/en#/',
  'MEDGEN': 'http://identifiers.org/medgen/',
  'MESH': 'http://identifiers.org/mesh/',
  'MONDO': 'http://purl.obolibrary.org/obo/MONDO_',
  'MedDRA': 'http://identifiers.org/meddra/',
  'NCBITaxon': 'http://purl.obolibrary.org/obo/NCBITaxon_',
  'NCIT': 'http://purl.obolibrary.org/obo/NCIT_',
  'OMIM': 'https://omim.org/entry/',
  'OMIMPS': 'https://omim.org/phenotypicSeries/PS',
  'Orphanet': 'http://www.orpha.net/ORDO/Orphanet_',
  'SCTID': 'http://identifiers.org/snomedct/',
  'UMLS': 'http://linkedlifedata.com/resource/umls/id/'

In [37]:
with open("metadata.yaml", "w") as f:
    yaml.dump(meta, f)

In [25]:
! sssom parse --help

Usage: sssom parse [OPTIONS] INPUT

  Parse a file in one of the supported formats (such as obographs) into an
  SSSOM TSV file.

Options:
  -I, --input-format TEXT         The string denoting the input format, e.g.
                                  tsv,rdf,owl,alignment-api-xml,obographs-
                                  json,json
  -m, --metadata PATH             The path to a file containing the sssom
                                  metadata (including prefix_map) to be used.
  -C, --prefix-map-mode [metadata_only|sssom_default_only|merged]
                                  Defines whether the prefix map in the
                                  metadata should be extended or replaced with
                                  the SSSOM default prefix map. Must be one of
                                  metadata_only, sssom_default_only, merged
                                  [default: metadata_only; required]
  -p, --clean-prefixes            If True (default), records with unknow

In [39]:
%%time
! sssom parse {ofn_json} -I obographs-json -o boomer.sssom.tsv -m metadata.yaml

In [40]:
! sssom diff --help

Usage: sssom diff [OPTIONS] INPUTS...

  Compare two SSSOM files.

  The output is a new SSSOM file with the union of all mappings, and injected
  comments indicating uniqueness to set1 or set2.

Options:
  -o, --output FILENAME  Path of SSSOM output file.
  --help                 Show this message and exit.


In [42]:
%%time
! sssom diff {input_sssom} boomer.sssom.tsv -o in_out_diff.sssom.tsv

CPU times: user 2.45 s, sys: 656 ms, total: 3.11 s
Wall time: 3min 26s


In [43]:
comparison_msdf = parse_sssom_table("in_out_diff.sssom.tsv")

In [44]:
comparison_msdf.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,confidence,comment
0,DOID:0001816,,owl:equivalentClass,MONDO:0016982,semapv:UnspecifiedMatching,NaN,UNIQUE_2
1,DOID:0002116,,owl:equivalentClass,MONDO:0005085,semapv:UnspecifiedMatching,NaN,UNIQUE_2
2,DOID:0014667,,owl:equivalentClass,MONDO:0005066,semapv:UnspecifiedMatching,NaN,UNIQUE_2
3,DOID:0040001,,owl:equivalentClass,MONDO:0025517,semapv:UnspecifiedMatching,NaN,UNIQUE_2
4,DOID:0040002,,owl:equivalentClass,MONDO:0025518,semapv:UnspecifiedMatching,NaN,UNIQUE_2


In [46]:
comparison_msdf.df["comment"].value_counts()

COMMON_TO_BOTH    139564
UNIQUE_2           69782
UNIQUE_1            1715
Name: comment, dtype: int64